In [1]:
import pyspark
pyspark.__version__

'3.1.1'

In [2]:
import os

In [3]:
# # setup arguments
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12-3.1.1.jar'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.11:2.1.0 pyspark-shell'

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        getOrCreate()

In [5]:
spark

In [6]:
from ast import literal_eval

In [7]:
# # default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
# KAFKA_BOOTSTRAP_SERVERS_CONS = 'kafka:9092'
# KAFKA_TOPIC_NAME_CONS = 'sales-topic'
# df = spark.readStream \
#      .format("kafka") \
#      .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
#      .option("subscribe", KAFKA_TOPIC_NAME_CONS) \
#      .option("startingOffsets", "earliest") \
#      .option("failOnDataLoss" ,"false")\
#      .load()

In [8]:
import json
from kafka import KafkaConsumer

topic = 'dev.vm-dev-tfm-uoc.crm-topic'
# To consume latest messages and auto-commit offsets
consumer = KafkaConsumer(topic,
                         bootstrap_servers=['broker:29092'])
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    print ("%s:%d:%d: key=%s val=%s " % (message.topic, message.partition,
                                         message.offset, message.key, message.value))

KeyboardInterrupt: 

In [ ]:
# consume earliest available messages, don't commit offsets
KafkaConsumer(auto_offset_reset='earliest', enable_auto_commit=False)

In [13]:
""" Read test data from Kafka to ensure producer and broker are working """
import json
from kafka import KafkaConsumer


topic = 'dev.vm-dev-tfm-uoc.crm-topic'

# To consume latest messages and auto-commit offsets
consumer = KafkaConsumer(topic,
                         bootstrap_servers=['broker:29092'])

data = []
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    data.append(message)
    print ("%s:%d:%d: key=%s val=%s " % (message.topic, message.partition,
                                         message.offset, message.key, message.value))

# consume earliest available messages, don't commit offsets
KafkaConsumer(auto_offset_reset='smallest', enable_auto_commit=False)

# consume json messages
KafkaConsumer(value_deserializer=lambda m: json.loads(m.decode('ascii')))


# StopIteration if no message after 1sec
KafkaConsumer(consumer_timeout_ms=1000)

dev.vm-dev-tfm-uoc.crm-topic:0:505: key=b'{"shop": "Luigis Pizza"}' val=b'{"id": 62, "shop": "Luigis Pizza", "created_at": "08:38:15 PM", "name": "Fabio Prieto Barba", "phoneNumber": "+34724119042", "address": "Calle Mercedes Mariscal 59 Puerta 8 \\nLa Coru\\u00f1a, 78869", "pizzas": [{"pizzaName": "Carbonara", "additionalToppings": ["onion", "olives", "strawberry", "green peppers"]}, {"pizzaName": "Mari & Monti", "additionalToppings": ["tuna", "onion"]}, {"pizzaName": "Salami", "additionalToppings": ["olives", "garlic", "garlic", "tomato"]}, {"pizzaName": "Salami", "additionalToppings": ["olives", "anchovies", "green peppers"]}, {"pizzaName": "Hawaiiana", "additionalToppings": ["anchovies", "tomato", "salami"]}, {"pizzaName": "Diavola", "additionalToppings": ["anchovies", "blue cheese", "tuna", "ham"]}, {"pizzaName": "Mari & Monti", "additionalToppings": ["olives", "strawberry", "tomato"]}, {"pizzaName": "Capricciosa", "additionalToppings": ["pineapple", "anchovies", "banana"]}, {"piz

KeyboardInterrupt: 

In [ ]:

# df1 = df.selectExpr( "CAST(value AS STRING)")

# df1.registerTempTable("test")


# # Deserilzing the Avro code function

# from pyspark.sql.column import Column, _to_java_column 
# def from_avro(col): 
#      jsonFormatSchema = """
#                     {
#                      "type": "record",
#                      "name": "struct",
#                      "fields": [
#                        {"name": "col1", "type": "long"},
#                        {"name": "col2", "type": "string"}
#                                 ]
#                      }"""
#     sc = SparkContext._active_spark_context 
#     avro = sc._jvm.org.apache.spark.sql.avro
#     f = getattr(getattr(avro, "package$"), "MODULE$").from_avro
#     return Column(f(_to_java_column(col), jsonFormatSchema))


# spark.udf.register("JsonformatterWithPython", from_avro)

# squared_udf = udf(from_avro)
# df1 = spark.table("test")
# df2 = df1.select(squared_udf("value"))

# #  Declaring the Readstream Schema DataFrame :

# df2.coalesce(1).writeStream \
#    .format("parquet") \
#    .option("checkpointLocation","/path/chk31") \
#    .outputMode("append") \
#    .start("/path/stream/tgt31")


# ssc.awaitTermination()

In [10]:
consumer.metrics()

{'kafka-metrics-count': {'count': 63.0},
 'consumer-metrics': {'connection-close-rate': 0.0,
  'connection-creation-rate': 0.0,
  'select-rate': 2.584675217846816,
  'io-wait-time-ns-avg': 245340508.71928534,
  'io-wait-ratio': 0.6341251993636998,
  'io-time-ns-avg': 247262.41827011108,
  'io-ratio': 0.0006390987286306676,
  'connection-count': 1.0,
  'network-io-rate': 2.58468274422033,
  'outgoing-byte-rate': 195.1476006530151,
  'request-rate': 1.2923679019244423,
  'request-size-avg': 151.0,
  'request-size-max': 151.0,
  'incoming-byte-rate': 267.9451679672966,
  'response-rate': 1.2923397665943988,
  'request-latency-avg': 491.2629872560501,
  'request-latency-max': 501.6303062438965},
 'consumer-node-metrics.node-bootstrap-0': {'outgoing-byte-rate': 0.0,
  'request-rate': 0.0,
  'request-size-avg': 0.0,
  'request-size-max': -inf,
  'incoming-byte-rate': 0.0,
  'response-rate': 0.0,
  'request-latency-avg': 0.0,
  'request-latency-max': -inf},
 'consumer-fetch-manager-metrics': 

In [12]:
consumer.partitions_for_topic(topic)

{0, 1, 2, 3}

In [18]:
data[0]['value']

TypeError: tuple indices must be integers or slices, not str